## Predictive Modeling on Starcraft Data
This notebook aims to perform specific classification methods to predict LeagueIndex. As LeagueIndex contains values from 1 to 8, certain methods don't really make sense in the context of this problem statement. For example, logistic regression is a binary classifier, and this wouldn't be applicable for the problem statement. Additionally, the presence of missing data for the LeagueIndex 8 players means that there must also be some modifications to the data or else some methods won't work at all. Upon researching potential methods, I settled on two for the sake of this assessment: one "simple" and one more complex. I chose a Naive Bayes classification as well as XGBoost, as those were the ones that seemed most appealing while also being quite different in methodology. 

### Filling in missing data
With missing data points, I attempt to fill in the gaps in order to make the most accurate model. For age, it seems as though the ages stay relatively the same throughout all LeagueIndices, so I will simply take the median age to fill in the missing values. However, for the other features that have missing features it seems that there is a linear or exponential growth as the LeagueIndex increases. For that, I will use different methods to fill in the gap. 

In [14]:
import numpy as np 
import pandas as pd
import load_data
from sklearn.linear_model import LinearRegression

#get data from local machine
spd = load_data.load_everything()
spd = spd.apply(pd.to_numeric, downcast = 'float')

#fill age with median age 
spd['Age'] = spd['Age'].fillna(spd['Age'].median())

#fill TotalHours with 

#spd = spd.to_numpy()


C:\Users\MarkLee\AppData\Local\Temp\ipykernel_3576\2557462944.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop('TotalHours', axis = 1, inplace = True)
C:\Users\MarkLee\AppData\Local\Temp\ipykernel_3576\2557462944.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop("TotalHours", axis = 1, inplace = True)
C:\Users\MarkLee\AppData\Local\Temp\ipykernel_3576\2557462944.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

### Naive Bayes Multinomial Classification

In [5]:

from sklearn.naive_bayes import MultinomialNB

np.random.seed(1)

#get data from local machine
spd = load_data.load_everything()
spd = spd.to_numpy()
# #split data into target x and y 
# Y = spd[:,1] 
X = spd[:,2:-1]
# # print(X)
# # print(Y)

# # create and train the model
# model = MultinomialNB(alpha = 1) 
# model.fit(X,Y) 
# Y_pred = model.predict(X) 
# acc_train = model.score(X,Y) 
# print("Accuracy = %0.4f " % acc_train)


In [7]:
from sklearn.impute import KNNImputer
fea_transformer = KNNImputer(n_neighbors = 3)
values = fea_transformer.fit_tansofrm


AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations